# Black and White Image Colorization

Bu projede siyah beyaz resimleri renklendiriyoruz.<br>

Bu projenin yapımında <a href="https://www.youtube.com/watch?v=gAmskBNz_Vc" target=_blank>bu videodan</a> faydalanılmıştır.<br><br>

![Einstein](../Data/einstein.png)

In [1]:
# Gerekli kütüphaneleri import ediyoruz.
import numpy as np
import argparse
import cv2
import os

## Modeli Yükleme

In [2]:
color_dir = "./colorize/"
prototx = os.path.join(color_dir, r"model/colorization_deploy_v2.prototxt")
points = os.path.join(color_dir, r"model/pts_in_hull.npy")
model = os.path.join(color_dir, r"model/colorization_release_v2.caffemodel")

In [3]:
net = cv2.dnn.readNetFromCaffe(prototx, model)
pts = np.load(points)

In [4]:
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

## Resmi Renklendirme

In [8]:
# Renklendirilecek resmin belirlenmesi
resim="colorize/images/einstein.jpg"
image = cv2.imread(resim)
scaled = image.astype("float32") / 255.0

#Resim renk modu değişiyor RGB -> LAB
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

In [9]:
resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

In [11]:
# Renkli resim oluşturuluyor
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))
L = cv2.split(lab)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

# Resim tekrar RGB renk moduna dönüştürülüyor
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)
colorized = (255 * colorized).astype("uint8")

cv2.imshow("Original", image)
cv2.imshow("Colorized", colorized)
cv2.waitKey()
cv2.destroyAllWindows()

## Sonuç:
Bu projede siyah beyaz bir resmi hazır eğitilmiş bir model kullanarak renklendirdik. Model resmi ilk önce RGB renk modundan L*a*b renk moduna dönüştürüyor. Kısaca Lab renk modu olarak bilinen bu yöntemle renkler L(siyah - beyaz tonu), a(kırmızı - yeşil tonu), b(sarı - mavi tonu) ifade etmektedir. Model girdi olarak L katmanı, çıktı olark a+b katmanı verilerek eğitilmiş ve bu şekilde resimler renklendirilmiştir.
<br><br> Bu projede oluşturulan uygulamaya <a href="https://huggingface.co/spaces/abdullatifkaban/Colorize" target=_blank>
<u>Hugging Face</u></a> üzerinden ulaşılabilir.